In [1]:
%load_ext autoreload
%autoreload 2


In [11]:
from dotenv import load_dotenv
from typing import Annotated
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from agent.knowledge_base import ask_doordash_question
import nest_asyncio
nest_asyncio.apply()
load_dotenv(override=True)

True

In [3]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

# llm = init_chat_model("openai/gpt-oss-20b", model_provider="groq",)
llm = init_chat_model("models/gemini-2.5-flash", model_provider="google_genai")

out = llm.invoke("hi")
print(out)

content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--d1c4d5f3-c559-4cc0-904d-0e73d19946dd-0' usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 449, 'input_token_details': {'cache_read': 0}}


In [4]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from datetime import datetime
prompt = ChatPromptTemplate.from_messages([
    ("system", "always return yes boss after anwering the question"),
    ("user", "{input}")
])


In [5]:
# @tool
# def add(a: float, b: float) -> float:
#     """always use this tool to add two numbers only"""
#     return a + b
from rich import print as rprint
@tool
def knowledge_base_query(query: str) -> str:
    """Queries the DoorDash knowledge base. 
    Always use this tool to answer questions related to DoorDash policies, procedures, and FAQs.
    """
    result = ask_doordash_question(query)
    rprint("knowledge base result:", result)
    return result

tools = [ knowledge_base_query]
llm = llm.bind_tools(tools)

In [ ]:
tool = knowledge_base_query
out = tool.invoke({"query": "i live in australia and im an independent contractor for doordash, i really need to know and understand the payment dispute process"})



INFO:agent.knowledge_base:Query engine successfully initialized.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"


knowledge base result:
Response(
    response='As an independent contractor for DoorDash in Australia, the payment dispute process depends on 
whether the issue stems from a service failure on your part or a failure by DoorDash to remit payment.\n\nIf a 
service failure occurs, which may result in you not being entitled to payment, DoorDash will make an initial 
determination based on evidence from all involved parties, including the consumer and merchant. You have the right 
to challenge this determination by notifying DoorDash in writing through help.doordash.com. Your notification 
should include any documents or information that support your challenge, providing DoorDash an opportunity to 
resolve the dispute.\n\nIf DoorDash fails to remit payment to you in a timely or accurate manner, you have the 
right to seek proper payment through legal avenues. Before pursuing such means, you must first inform DoorDash in 
writing at help.doordash.com of the payment failure and allow them a reasonable opportunity to rectify it. Should 
you prevail in seeking proper payment, you would be entitled to recover reasonable costs incurred during that 
process. This specific process does not apply to issues related to cash-based contracted service opportunities.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='d3aee029-4781-4066-b19b-c4b9bd20486e',
                embedding=None,
                metadata={
                    'page_number': 26,
                    'file_name': 'Independent Contractor Agreement for NEW ZEALAND.pdf',
                    'page_links': [],
                    'source_link': 'https://example.com/policies/4783',
                    'questions_this_excerpt_can_answer': "**Higher-level summaries of surrounding context:**\n\n1. 
The context appears to be a contract between DoorDash and an independent contractor, outlining the terms and 
conditions of their agreement, including a mutual arbitration provision.\n2. The contract is specific to New 
Zealand, as indicated by the file name and page number.\n3. The contract includes a provision for electronic copies
to be sent to a specific email address (legal@doordash.com) and specifies that any demand for arbitration must be 
delivered to the contractor's email address.\n\n**Questions that this context can provide specific answers 
to:**\n\n1. What is the specific email address to which any demand for arbitration by DoorDash must be delivered, 
as per the contract?\n\nThis question can be answered directly from the context, as it is explicitly stated in the 
contract.\n\n**Additional questions that can be generated using the higher-level summaries:**\n\n1. What are the 
specific requirements for arbitration under the contract, and how do they differ from traditional court 
proceedings?\n2. How does the contract address the issue of confidentiality and proprietary information in the 
context of arbitration?\n3. What are the implications of the contract's mutual arbitration provision for the 
contractor's status as an independent contractor?\n4. How does the contract's provision for electronic copies to be
sent to a specific email address (legal@doordash.com) relate to the overall structure and purpose of the 
contract?\n5. What are the specific laws and regulations that govern the contract, and how do they impact the terms
and conditions outlined in the contract?"
                },
                excluded_embed_metadata_keys=['file_name', 'page_number'],
                excluded_llm_metadata_keys=['file_name', 'page_number'],
                relationships={
                    <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(
                        node_id='5d4a8899-133f-4419-8c5a-65ba291ae998',
                        node_type='4',
                        metadata={
                            'page_number': 26,
                            'file_name': 'Independent Contractor Agreement for NEW ZEALAND.pdf',
                   

In [32]:
print(f"Retrieved {len(out.source_nodes)} chunks")

for i, node in enumerate(out.source_nodes, start=1):
    print(f"\n--- Chunk {i} | Score: {node.score} ---\n")
    print(node.node.get_content()[:300])  # preview first 300 chars

# print(f"Retrieved {len(out)} chunks")

# for i, node in enumerate(out, start=1):
#     print(f"\n--- Chunk {i} | Score: {node.score} ---")
#     print(f"Source: {node.node.metadata.get('file_name', 'N/A')}")
#     print(node.node.get_content()[:300] + "...")


Retrieved 20 chunks

--- Chunk 1 | Score: 0.7431849634283489 ---

and an electronic copy must also be sent to legal@doordash.com. Any demand for arbitration by DoorDash must be delivered to the email address of Contractor, as supplied to DoorDash by Contractor.

# 4.

Contractor agrees and acknowledges that entering into this arbitration agreement does not change 

--- Chunk 2 | Score: 0.7431849634283489 ---

and an electronic copy must also be sent to legal@doordash.com. Any demand for arbitration by DoorDash must be delivered to the email address of Contractor, as supplied to DoorDash by Contractor.

# 4.

Contractor agrees and acknowledges that entering into this arbitration agreement does not change 

--- Chunk 3 | Score: 0.7421247489025671 ---

# Mutual Arbitration Provision

1. Any referral to arbitration must be done in accordance with clause 24.2(c)(3) above.
2. The arbitration shall be heard by one arbitrator selected by DoorDash, or if such a selection is not made by DoorDash

In [ ]:
matches = []

for node in out.source_nodes:
    text = node.node.text if hasattr(node, "node") else node.text
    if "payment disputes" in text.lower():
        matches.append(text)

if matches:
    print(f"✅ Found {len(matches)} matching chunks:\n")
    for i, match in enumerate(matches, start=1):
        print(f"--- Match {i} ---\n{match}\n")
else:
    print("❌ Not found in the retrieved chunks")


In [45]:
from IPython.display import Markdown

Markdown(out.response)


As an independent contractor for DoorDash in Australia, the payment dispute process depends on whether the issue stems from a service failure on your part or a failure by DoorDash to remit payment.

If a service failure occurs, which may result in you not being entitled to payment, DoorDash will make an initial determination based on evidence from all involved parties, including the consumer and merchant. You have the right to challenge this determination by notifying DoorDash in writing through help.doordash.com. Your notification should include any documents or information that support your challenge, providing DoorDash an opportunity to resolve the dispute.

If DoorDash fails to remit payment to you in a timely or accurate manner, you have the right to seek proper payment through legal avenues. Before pursuing such means, you must first inform DoorDash in writing at help.doordash.com of the payment failure and allow them a reasonable opportunity to rectify it. Should you prevail in seeking proper payment, you would be entitled to recover reasonable costs incurred during that process. This specific process does not apply to issues related to cash-based contracted service opportunities.

In [6]:
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

In [19]:
class State(TypedDict):
    """
    The state of the chatbot.
    messages is a list of messages in the conversation.

    """
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    messages = state["messages"]
    print(f"Chatbot invoked. Current conversation length: {len(messages)}")

    prompt = ChatPromptTemplate.from_messages([
        ("system", "Always return the output from the tools as is with all the details very important "),
        MessagesPlaceholder(variable_name="messages"),
    ])
    response = llm.invoke(prompt.format(messages=messages))
    return {"messages": [response]}


def prune_history(state: State) -> State:
    messages = state["messages"]
    print(f"Pruning history. Current length: {len(messages)}")
    max_messages = 10
    if len(messages) > max_messages:
        # Keep the last 15 messages (a small offset to avoid immediate re-pruning)
        pruned_messages = messages[-(max_messages - 5):]
        # You might also want to keep the system message
        # pruned_messages = [messages[0]] + messages[-(max_messages - 1):]
        print(f"Pruned history to length: {len(pruned_messages)}")
        return {"messages": pruned_messages}
    return state


builder = StateGraph(State)

builder.add_node("Chatbot", chatbot)
builder.add_node("tools", ToolNode(tools=tools))
builder.add_node("prune_history", prune_history)

builder.add_edge(START, "Chatbot")
builder.add_conditional_edges("Chatbot", tools_condition, {"tools": "tools", END: END})
builder.add_edge("tools", "Chatbot")
builder.add_edge("Chatbot", END)
graph = builder.compile(checkpointer=memory)


In [8]:

print(graph.get_graph().draw_mermaid())


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	Chatbot(Chatbot)
	tools(tools)
	prune_history(prune_history)
	__end__(<p>__end__</p>)
	Chatbot -.-> __end__;
	Chatbot -.-> tools;
	__start__ --> Chatbot;
	tools --> Chatbot;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [9]:
config = {"configurable": {"thread_id": "15", "max_iterations": 3, "verbose": True, "recursion_limit": 3}}

In [26]:
del memory.storage["15"]

In [27]:
out = graph.invoke({"messages": [HumanMessage(content="how does refunds work?")]}, config=config)
print(out["messages"][-1].content)

Chatbot invoked. Current conversation length: 1


INFO:agent.knowledge_base:Query engine successfully initialized.
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: You are an expert DoorDash customer support assistant.
You answer using only the retrieved DoorDash documentation and metadata.

How to use metadata (very important):
- Each retrieved chunk may include metadata fields:
  - `source_link`: the canonical source URL for that document.
  - `page_links`: a list of link objects like {"text": "...", "url": "..."} that were embedded in the page.
- In your answer body, embed ONLY the page links that are directly relevant to what you are explaining.
  - Use the link's `text` as the anchor and `url` as the target, e.g. [Change payout method](https://...).
  - Do NOT dump all page_links. Use them sparingly and only when they help the user complete a task.
- At the end, ALWAYS include a **Sources** section built from `source_link` values found across the retrieved chunks.
  - Deduplicate URLs; list each once as

knowledge base result: I’m sorry, but the information you’re looking for isn’t included in the excerpt I have.  

**Sources**  
- https://example.com/policies/6767
Chatbot invoked. Current conversation length: 3
I’m sorry, but the information you’re looking for isn’t included in the excerpt I have.

**Sources**
- https://example.com/policies/6767


In [28]:
from rich import print as rprint
print(len(out["messages"]))
rprint(out["messages"])

4


[
    HumanMessage(
        content='how does refunds work?',
        additional_kwargs={},
        response_metadata={},
        id='a24c50cc-bb27-4a66-980c-35737c310a0f'
    ),
    AIMessage(
        content='',
        additional_kwargs={
            'function_call': {'name': 'knowledge_base_query', 'arguments': '{"query": "how do refunds work?"}'}
        },
        response_metadata={
            'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
            'finish_reason': 'STOP',
            'safety_ratings': []
        },
        id='run--82daaca2-30fc-4a89-b2f3-5f54e3fd87c2-0',
        tool_calls=[
            {
                'name': 'knowledge_base_query',
                'args': {'query': 'how do refunds work?'},
                'id': '8b653316-8380-4237-8500-dbf477a4721c',
                'type': 'tool_call'
            }
        ],
        usage_metadata={
            'input_tokens': 89,
            'output_tokens': 21,
            'total_tokens': 194,
            'input_token_details': {'cache_read': 0}
        }
    ),
    ToolMessage(
        content='I’m sorry, but the information you’re looking for isn’t included in the excerpt I have.  
\n\n**Sources**  \n- https://example.com/policies/6767',
        name='knowledge_base_query',
        id='59e07ef6-9533-4ed3-af22-8d87db78d221',
        tool_call_id='8b653316-8380-4237-8500-dbf477a4721c'
    ),
    AIMessage(
        content='I’m sorry, but the information you’re looking for isn’t included in the excerpt I 
have.\n\n**Sources**\n- https://example.com/policies/6767',
        additional_kwargs={},
        response_metadata={
            'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
            'finish_reason': 'STOP',
            'safety_ratings': []
        },
        id='run--cfe0b75d-2d06-4d1a-9d5f-597e4f8883ca-0',
        usage_metadata={
            'input_tokens': 163,
            'output_tokens': 41,
            'total_tokens': 384,
            'input_token_details': {'cache_read': 0}
        }
    )
]

In [ ]:
del memory.storage["15"]
# rprint(memory.storage["5"])